<a href="https://colab.research.google.com/github/kHarshit/Colorization-of-paintings-GANs/blob/main/ML_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch 
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob
import PIL.Image
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os

plt.style.use('seaborn')

<ipython-input-3-3c392b102641>:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
os.listdir('/content/drive/MyDrive/ML_Project/dataset/')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Hyperparameters

TRAIN_SPLIT = 0.7
VALID_SPLIT = 0.2
TEST_SPLIT = 0.1
BATCH_SIZE = 32
LR = 0.001
EPOCHS = 20

In [4]:
"""
Need some modifications. Will fix later. -Harshit
"""
class Colorization(Dataset):
  def __init__(self,imagePath):
    self.imagePath = imagePath
    self.transform_gray = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5,),(0.5,0.5,0.5))
    ]) 
    self.transform_color = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ])
  def __len__(self):
    return len(os.listdir(self.imagePath))
  def __getitem__(self, index):
    imagepath = os.path.join(self.imagePath,os.listdir(self.imagePath)[index])
    color_image = PIL.Image.open(imagepath).convert('RGB')
    color_img = self.transform_color(color_image)
    gray_img = self.transform_gray(color_image)
    return gray_img,color_img

In [ ]:
final_dataset = DataLoader(Colorization('/content/drive/MyDrive/ML_Project/dataset'),batch_size=BATCH_SIZE,shuffle=True)

# splitting into training testing validation dataset
train_ratio_size = int(TRAIN_SPLIT*(len(final_dataset.dataset)))
valid_ratio_size = int(VALID_SPLIT*(len(final_dataset.dataset)))
test_ratio_size = int(TEST_SPLIT*(len(final_dataset.dataset)))

train_set,valid_set,test_set = random_split(final_dataset.dataset,[train_ratio_size,valid_ratio_size,test_ratio_size])

datasets = {
    'train': train_set,
    'valid': valid_set,
    'test': test_set
}

loaders = {x: DataLoader(datasets[x], batch_size=32, shuffle=True)
              for x in ['train', 'valid', 'test']}

dataset_sizes = {x: len(datasets[x]) for x in ['train', 'valid', 'test']}
print(dataset_sizes)

# for batch_id,(X,y) in enumerate(final_dataset):
#   plt.subplot(1,1,1)
#   plt.tight_layout()
#   plt.imshow(y[0].numpy().transpose((1, 2, 0)))
#   break
# just for checking
# for batch_id,(X,y) in enumerate(train_dataset):
#   print(X.shape,y.shape)
#   plt.subplot(1,3,1)
#   plt.tight_layout()
#   plt.imshow(X[0].numpy().transpose((1, 2, 0)))
#   break

# for batch_id,(X,y) in enumerate(test_dataset):
#   print(X.shape,y.shape)
#   plt.subplot(1,3,2)
#   plt.tight_layout()
#   plt.imshow(X[0].numpy().transpose((1, 2, 0)))
#   break

# for batch_id,(X,y) in enumerate(valid_dataset):
#   print(X.shape,y.shape)
#   plt.subplot(1,3,3)
#   plt.tight_layout()
#   plt.imshow(X[0].numpy().transpose((1, 2, 0)))
#   break

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x

In [ ]:
model = Model()
model.to(device)

In [ ]:
from torchsummary import summary
summary(model, input_size=(300,))

In [ ]:
# Set the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    train_loss_list, valid_loss_list = [], []
    train_acc_list, valid_acc_list = [], []

    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss and accuracy
        train_loss = 0.0
        valid_loss = 0.0
        train_correct = 0
        valid_correct = 0

        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            data, target = data.to(device), target.to(device)
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
            ## calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == target).sum().item()

        ######################    
        # validate the model #
        ######################
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(loaders['valid']):
                # move to GPU
                data, target = data.to(device), target.to(device)
                ## update the average validation loss
                outputs = model(data)
                loss = criterion(outputs, target)
                valid_loss += loss.item() * data.size(0)
                ## calculate validation accuracy
                _, predicted = torch.max(outputs, 1)
                valid_correct += (predicted == target).sum().item()

        # print training/validation/test statistics 
        # calculate average losses
        train_loss = train_loss/len(loaders['train'].dataset)
        valid_loss = valid_loss/len(loaders['valid'].dataset)

        train_acc = 100. * train_correct/len(loaders['train'].dataset)
        valid_acc = 100. * valid_correct/len(loaders['valid'].dataset)

        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)
        train_acc_list.append(train_acc)
        valid_acc_list.append(valid_acc)

        print('Epoch: {} \tTraining Loss: {:.6f} \tTraining Accuracy: {:.2f}% \tValidation Loss: {:.6f} \tValidation Accuracy: {:.2f}%'.format(
            epoch, train_loss, train_acc, valid_loss, valid_acc))

        ## save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

    # return trained model
    return train_loss_list, valid_loss_list, train_acc_list, valid_acc_list

In [ ]:
# train the model
train_loss_list, valid_loss_list, train_acc_list, valid_acc_list = train(EPOCHS, loaders, model, optimizer, criterion, 'model.pt')

In [ ]:
# load best validation model
model.load_state_dict(torch.load('model.pt'))